In [1]:
from distutils.log import info
import datetime
import pandas as pd
import numpy as np
from SQL import SQL_TabellenLadenBearbeiten as SQL
from SQL import createnewTable

##TODO: SAFE BACKUP LT22 BEFORE ANYTHING



#dfMitarbeiter = SQL.sql_datenTabelleLaden(SQL.tabellemitarbeiter)
import pandas as pd

# Read the Excel file into a DataFrame
dflt22 = pd.read_excel('lt22.xlsx')

dfStammdaten = SQL.sql_Stammdaten()
dfOrders = SQL.sql_datenTabelleLaden(SQL.tabelle_DepotDEBYKNOrders)
dfMitarbeiter = SQL.sql_datenTabelleLaden(SQL.tabellemitarbeiter)

dflt22.set_index('Transfer Order Number', inplace=True)

# Weil ich nunmal ein Excel Idiot bi

dflt22.columns = ['MaterialNumber','C','D','E','F','G','H','I','J','Confirmation time','Confirmation date','M','MitarbeiterCreateTO','MitarbeiterConfirmTO','P','Creation Time','R','Creation Date','T','DestStorType','DestBin','W','X','Y','Z','AA','AB','AC']
dflt22['Ziel'] = dflt22['DestStorType'].str[:2]
dflt22['Quelle'] = dflt22['E'].str[:2]
dflt22['Pick Datum'] = dflt22['Confirmation date'].dt.strftime('%m/%d/%y')
#dflt22['K'] = dflt22['K'].astype(str)
dflt22['MaterialNumber'] = dflt22['MaterialNumber'].astype(str)
dflt22['Confirmation time'] = pd.to_datetime(dflt22['Confirmation time'], format='%H:%M:%S')
dflt22['Pick Zeit'] = dflt22['Confirmation time']
dflt22['Confirmation date'] = pd.to_datetime(dflt22['Confirmation date'])


        

In [2]:
dfMitarbeiter.head() 

,One ID,Name,Funktion,Unternehmen,Fachbereich
0,85139004,Bernd Schwing,Operativ,BAT,Super-Depot
1,85139053,Thair Abdulabbas,Operativ,BAT,Super-Depot
2,85138910,Patrick Keiner,Operativ,BAT,Super-Depot
3,85139259,Maximilian Kolb,Operativ,BAT,Super-Depot
4,85139619,Manuela Lautner,Operativ,BAT,Super-Depot


In [3]:
#dfUser.head()

In [4]:
#Stammdaten 
dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]
dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]   
def f_CS(row):
    try:
        if row.UnitOfMeasure == 'CS':          
            return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
def f_PAL(row):
    try:
        if row.UnitOfMeasure == 'D97':
            return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
def f_OUT(row):
    try:
        if row.UnitOfMeasure == 'OUT':
            return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
dfStammdaten['OUT'] = dfStammdaten.apply(f_OUT,axis=1)
dfStammdaten['CS'] = dfStammdaten.apply(f_CS,axis=1)
dfStammdaten['PAL'] = dfStammdaten.apply(f_PAL,axis=1)


In [5]:
# set dflt22 'SuperDepot' to true if the value from dflt22[DestStorBin] exists in dfOrders[SapOrderNumber]
dflt22['SuperDepot'] = dflt22['DestBin'].isin(dfOrders['SapOrderNumber'])


In [6]:
dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dflt22 = pd.merge(dflt22, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

In [7]:



# print('pickschleife')
# #Schleife erst Picks Berechnen -> Dann Umlagerungen ermitteln
for each in dflt22.index:
    if dflt22.loc[each, 'E'] == 'TN1' and dflt22.loc[each, 'DestStorType'] == '916' and dflt22.loc[each, 'DestStorType'] == '916':
        dflt22.loc[each, 'PICKS'] = dflt22.loc[each, 'H'] 
        dflt22.loc[each, 'Picks OUT'] = dflt22.loc[each, 'H'] 
        dflt22.loc[each, 'Pick Art'] = 'Stange'
    if dflt22.loc[each, 'E'] == 'SN1' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN2' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN3' and dflt22.loc[each, 'DestStorType'] == '916' or dflt22.loc[each, 'E'] == 'SN4' and dflt22.loc[each, 'DestStorType'] == '916':
        dflt22.loc[each, 'PICKS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'CS']
        dflt22.loc[each, 'Picks CS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'CS']
        dflt22.loc[each, 'Pick Art'] = 'Karton'
    if dflt22.loc[each, 'E'] == 'BS3' and dflt22.loc[each, 'DestStorType'] == '916':
        dflt22.loc[each, 'PICKS'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'PAL']
        dflt22.loc[each, 'PICKS PAL'] = (dflt22.loc[each, 'H'] * dflt22.loc[each, 'OUT']) / dflt22.loc[each, 'PAL']
        dflt22.loc[each, 'Pick Art'] = 'Palette'
    #Umlagerungen Ermitteln 
    if dflt22.loc[each, 'E'] == 'BS3' and dflt22.loc[each, 'Ziel'] == 'SN':
        dflt22.loc[each, 'Umlagerung'] = 1
        dflt22.loc[each, 'Art'] = 'Karton'
    if dflt22.loc[each, 'Quelle'] == 'RS' and dflt22.loc[each, 'Ziel'] == 'SN':
        dflt22.loc[each, 'Umlagerung'] = 1
        dflt22.loc[each, 'Art'] = 'Karton'
    if dflt22.loc[each, 'Quelle'] == 'SN' and dflt22.loc[each, 'Ziel'] == 'TN':
        dflt22.loc[each, 'Umlagerung'] = 1
        dflt22.loc[each, 'Art'] = 'Gebinde'



In [8]:
## Füge den Mitarbeiter hinzu
# if the value of dfMitarbeiter[OneID] exists in dflt22[MitarbeiterConfirmTO] then add the value of dfMitarbeiter[Mitarbeiter] to dflt22[MitarbeiterConfirmTO]
dflt22 = pd.merge(dflt22, dfMitarbeiter,left_on='MitarbeiterConfirmTO', right_on='One ID',how='left')
#fill none with 'Unbekannt'
dflt22['Name'] = dflt22['Name'].fillna('Unbekannt')

In [9]:
dflt22['LieferscheinErhalten'] = dfOrders['SapOrderNumber'].apply(lambda x: dfOrders.loc[dfOrders['SapOrderNumber'] == x]['CreatedTimestamp'].iloc[0])


In [11]:
# Filter by Creation date 07.12.2022
dflt22 = dflt22[dflt22['Confirmation date'] > '2021-12-07 00:00:00']


In [20]:
import plotly.express as px

# Lese die Daten aus der Tabelle ein und speichere sie in einem Pandas-DataFrame
df = dflt22

df['Creation Time'] = pd.to_datetime(df['Creation Time'])
df['Confirmation time'] = pd.to_datetime(df['Confirmation time'])

# Berechne die Dauer der Verfügbarkeit in Minuten und speichere das Ergebnis in einer neuen Spalte "Availability duration"
df['Availability duration'] = (df['Confirmation time'] - df['Creation Time']).dt.total_seconds() / 60

# Erstelle eine neue Spalte "Availability hour", die die Stunde der Verfügbarkeit enthält
df['Availability hour'] = df['Creation Time'].dt.hour

# Erstelle eine neue Spalte "Total picks", die die Summe der PICKS pro Stunde enthält
df_agg = df.groupby('Availability hour').agg({'PICKS': 'sum'}).reset_index()

df_agg = df_agg.merge(df[['Availability hour', 'Availability duration']], on='Availability hour', how='left')

# Erstelle das Chart mit Plotly
fig = px.bar(df_agg, x='Availability hour', y='PICKS', title='Verfügbarkeit von Positionen und PICKS pro Stunde',
             color='Availability duration', color_continuous_scale=px.colors.sequential.Plasma)

# Zeige das Chart an
fig.show()